<a href="https://colab.research.google.com/github/arielkeren/photo-organizer/blob/main/PhotoOrganizerDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install insightface onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 11.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 115.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.9 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp311-cp311-linux_x86_64.whl size=1064784 sha256=7b79a93c7b65df4e44e8efab7236d5e9446903ee9a370df0f787791073d4285b
  Stored in directory: /root/.cache/pip/wheels/27/d8/22/f52d858d16cd06e7b2e6aad34a1777dcfaf000be833bbf8146
Successfully built insightface


In [ ]:
import os
import numpy as np
from tqdm import tqdm
from PIL import Image
import cv2
from insightface.app import FaceAnalysis

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
app = FaceAnalysis(name='buffalo_l')
app.prepare(ctx_id=0)

DATA_FOLDER = "/content/drive/MyDrive/family_photos"

x = []
y = []

label_names = []
label_map = {}

for label_idx, person_name in enumerate(sorted(os.listdir(DATA_FOLDER))):
    person_folder = os.path.join(DATA_FOLDER, person_name)
    if not os.path.isdir(person_folder):
        continue

    label_map[label_idx] = person_name
    label_names.append(person_name)

    for file_name in tqdm(os.listdir(person_folder), desc=person_name):
        file_path = os.path.join(person_folder, file_name)
        if not file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        try:
            image = cv2.imread(file_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            faces = app.get(image)

            if len(faces) > 0:
                # Use the first face detected
                emb = faces[0].embedding
                x.append(emb)
                y.append(label_idx)
        except Exception as e:
            print(f"Error processing {file_path}: {e}")

x = np.array(x)
y = np.array(y)

print(f"Extracted {len(x)} embeddings.")

download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:03<00:00, 76086.21KB/s]
/usr/local/lib/python3.11/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:118: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


5: 100%|██████████| 50/50 [01:44<00:00,  2.09s/it]

Extracted 173 embeddings.


In [ ]:
np.save("embeddings.npy", x)
np.save("embedding_labels.npy", y)